In [79]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
import json
import math

# application config

In [80]:
EnableColorStream = False
ColorStreamResWidth = 1920
ColorStreamResHeight = 1080
ColorStreamFPS = 30

EnableInfraredStream = True
InfraredStreamResWidth = 1280
InfraredStreamResHeight = 720
InfraredStreamFPS = 30

EnableDepthStream = True
DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30

#cv2.namedWindow('Color image')
cv2.namedWindow('Infrared image')
cv2.namedWindow('Depth image')

maxDistance = 7000 # 1 meter

font = cv2.FONT_HERSHEY_TRIPLEX
textString = ''
colorFPS = (255,255,0)
colorDist = (0, 255,255)
colorRect = (0, 255,255)

colorLine = (0, 255, 0)
sizeLine = 3
colorLineP = (0, 255, 255)
sizeLineP = 1

# flags and global parameters

In [81]:
capture = False
point = False

# measure distance
paint = False
display = False
paintLTX = -1
paintLTY = -1
paintRDX = -1
paintRDY = -1
img = np.zeros((512,512,3), np.uint8)
click = False
img2 = np.zeros((512,512,3), np.uint8)

#development use
devX = -1
devY = -1

# RealSense camera config

In [82]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream is True:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableInfraredStream is True:
    config.enable_stream(
        rs.stream.infrared, InfraredStreamResWidth, InfraredStreamResHeight, rs.format.bgr8, InfraredStreamFPS)
    
if EnableDepthStream is True:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
cfg = pipeline.start(config)

# Alignment
if EnableColorStream is True:
    align_to = rs.stream.color
elif EnableInfraredStream is True:
    align_to = rs.stream.infrared

align = rs.align(align_to)
    
# Advanced settings
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 4)
depth_sensor.set_option(rs.option.enable_auto_exposure, 1)

# Get depth scale
scale = depth_sensor.get_depth_scale()
print("depth scale:" + str(scale))
# PointCloud settings
pc = rs.pointcloud()

depth scale:0.0010000000474974513


# color map function

In [83]:
def procColorMap(input_image):
    inp = input_image.copy()
    minNum, maxNum, minLoc, maxLoc = cv2.minMaxLoc(inp)

    if maxNum > maxDistance:
        maxNum = maxDistance
    
    scaleAlpha = 255 / maxNum
    inp = cv2.convertScaleAbs(inp, None, scaleAlpha, 0)
    depth_colormap = cv2.applyColorMap(inp, cv2.COLORMAP_JET)
    
    return depth_colormap

# screen capture function

In [84]:
def procScreenCapture():  
    global capture
    if capture is True:
        capture = False
        if EnableColorStream is True:
            fileName = "../imageCapture/color_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, color_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableInfraredStream is True:
            fileName = "../imageCapture/infrared_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, infrared_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableDepthStream is True:
            fileName = "../imageCapture/depth_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, depth_colormap, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


# measure distace and sectional drawing function

In [85]:
# mouse click function
def procMouse(event,x,y,flags,param):
    global paintLTX, paintLTY, paintRDX, paintRDY
    global click
    global img, img2
    
    if event is cv2.EVENT_LBUTTONDOWN:
        paintLTX = x
        paintLTY = y
        click = True
        img2 = img.copy()
        
    elif event is cv2.EVENT_MOUSEMOVE:
        if click is True:
            paintRDX = x
            paintRDY = y

    elif event is cv2.EVENT_LBUTTONUP:
        paintRDX = x
        paintRDY = y
        click = False

In [86]:
# sectional drawing function
def drawSect(secLTX, secLTY, secRDX, secRDY):
    global depth_image
    output = []
    
    xdiff = secRDX - secLTX
    ydiff = secLTY - secRDY
    
    if xdiff < ydiff:
        for i in range(0, ydiff):
            posX = math.floor(secLTX + i * xdiff / ydiff)
            posY = math.floor(secRDY + i)
            data = depth_image[posY, posX]
            output.append(data)
            print('x:' + str(posX) + ' y:' + str(posY) + ' z:' + str(data))
    else:
        for i in range(0, xdiff):
            posX = math.floor(secLTX + i)
            posY = math.floor(secRDY + i * ydiff / xdiff)
            data = depth_image[posY, posX]
            output.append(data)
            print('x:' + str(posX) + ' y:' + str(posY) + ' z:' + str(data))
    
    cv2.namedWindow('Sectional drawing')
    

In [87]:
# mouse paint function
def procPaint():
    global paint, display, click
    global paintLTX, paintLTY, paintRDX, paintRDY
    global img, img2
    
    if paint is True:
        img = infrared_image.copy()
        cv2.namedWindow('Display')
        cv2.setMouseCallback('Display', procMouse)
        cv2.imshow('Display', img)
        display = True
        paint = False
    
    if display is True:
        if click is True:
            img2 = img.copy()
            if paintLTX != -1 and paintRDX != -1:
                cv2.line(img2, (paintLTX,paintLTY), (paintRDX, paintRDY), colorLineP, sizeLineP)
            cv2.imshow('Display', img2)
        else:
            if paintLTX != -1 and paintRDX != -1:
                cv2.line(img, (paintLTX,paintLTY), (paintRDX, paintRDY), colorLine, sizeLine)
                textString = 'distance:'
                textX = int((paintLTX+paintRDX)/2)
                textY = int((paintLTY+paintRDY)/2)
                #cv2.rectangle(img, (textX-155, textY-35), (textX+155, textY+35), colorRect, -1)
                #cv2.rectangle(img, (textX-150, textY-30), (textX+150, textY+30), (255,255,255), -1)
                cv2.putText(img, textString, ( textX-100, textY), font, 1, colorDist, 1, cv2.LINE_AA)
                drawSect(paintLTX, paintLTY, paintRDX, paintRDY)
                paintLTX = -1
                paintLTY = -1
                paintRDX = -1
                paintRDY = -1


            cv2.imshow('Display', img)

In [88]:
# proc for development
def procDepthValue(event,x,y,flags,param):
    global devX, devY
    
    if event is cv2.EVENT_MOUSEMOVE:
        if x != 0 and y != 0:
            devX = x
            devY = y

# pointcloud function

In [89]:
def procPointCloud():
    global point
    
    if point is True:
        point = False
        points = pc.calculate(depth_frame)
        
        if EnableColorStream is True:
            pc.map_to(color_frame)
            
        if EnableInfraredStream is True:
            pc.map_to(infrared_frame)
            
        vertices = points.get_vertices()
        tex_coords = points.get_texture_coordinates()
        
        for i in range(0, points.size()):
            if vertices[i].z:
                print(vertices[i].x)
                print(vertices[i].y)
                print(vertices[i].z)
                break

# main loop

In [90]:
while True:
    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    
    # Aligh frames
    aligned_frames = align.proccess(frames)
    
    if EnableColorStream is True:
        color_frame = aligned_frames.get_color_frame()
        if not color_frame:
            continue
            
    if EnableInfraredStream is True:
        infrared_frame = aligned_frames.first(rs.stream.infrared)
        if not infrared_frame:
            continue
    
    if EnableDepthStream is True:
        depth_frame = aligned_frames.get_depth_frame()
        if not depth_frame:
            continue
    
    # Convert images to numpy arrays
    if EnableColorStream is True:
        color_image = np.asanyarray(color_frame.get_data())
    
    if EnableInfraredStream is True:
        infrared_image = np.asanyarray(infrared_frame.get_data())
    
    if EnableDepthStream is True:
        depth_image = np.asanyarray(depth_frame.get_data())
    
    # colorize depth image
    depth_colormap = procColorMap(depth_image)
    
    
    cv2.setMouseCallback('Depth image', procDepthValue)
   
    # Show images
    if EnableColorStream is True:
        cv2.imshow('Color image', color_image)
        
    if EnableInfraredStream is True:
        cv2.imshow('Infrared image', infrared_image)
    
    if EnableDepthStream is True:
        cv2.circle(depth_colormap, (devX,devY), 5, colorLineP, -1)
        textString = str(depth_image[devY, devX])
        cv2.putText(depth_colormap, textString, (devX-25, devY), font, 1, colorDist, 1, cv2.LINE_AA)
        cv2.imshow('Depth image', depth_colormap)
    
    # Screen capture
    procScreenCapture()
        
    # Read device temperature
    '''
    temp = depth_sensor.get_option(rs.option.projector_temperature)
    print("proj: " + str(temp))
    temp = depth_sensor.get_option(rs.option.asic_temperature)
    print("asic: " + str(temp))
    '''
    
    # Generate point cloud
    #procPointCloud()
    
    # Painit function
    procPaint()
        
    # Keyboard command
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c') or getKey is ord('C'):
        capture = True
    elif getKey is ord('q') or getKey is ord('Q'):
        break
    elif getKey is ord('p') or getKey is ord('P'):
        point = True
    elif getKey is ord('a') or getKey is ord('A'):
        paint = True
    elif getKey is ord('x') or getKey is ord('X'):
        cv2.destroyAllWindows()

x:502 y:377 z:8365
x:503 y:376 z:8365
x:504 y:376 z:8323
x:505 y:376 z:8323
x:506 y:375 z:8323
x:507 y:375 z:8323
x:508 y:375 z:8323
x:509 y:375 z:8323
x:510 y:374 z:8365
x:511 y:374 z:8365
x:512 y:374 z:8407
x:513 y:374 z:8407
x:514 y:373 z:8450
x:515 y:373 z:8493
x:516 y:373 z:8537
x:517 y:373 z:8537
x:518 y:372 z:8581
x:519 y:372 z:8581
x:520 y:372 z:8625
x:521 y:372 z:8625
x:522 y:371 z:8670
x:523 y:371 z:8670
x:524 y:371 z:8715
x:525 y:371 z:8715
x:526 y:370 z:8715
x:527 y:370 z:8715
x:528 y:370 z:8761
x:529 y:370 z:8761
x:530 y:369 z:8761
x:531 y:369 z:8761
x:532 y:369 z:8808
x:533 y:368 z:8808
x:534 y:368 z:8808
x:535 y:368 z:8808
x:536 y:368 z:8808
x:537 y:367 z:8808
x:538 y:367 z:0
x:539 y:367 z:0
x:540 y:367 z:13425
x:541 y:366 z:13757
x:542 y:366 z:13872
x:543 y:366 z:13872
x:544 y:366 z:13872
x:545 y:365 z:14228
x:546 y:365 z:13989
x:547 y:365 z:13872
x:548 y:365 z:13645
x:549 y:364 z:13645
x:550 y:364 z:13425
x:551 y:364 z:13211
x:552 y:364 z:13005
x:553 y:363 z:12707
x:55

In [91]:
# Stop streaming
cv2.destroyAllWindows()
pipeline.stop()